In [ ]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [ ]:
df_train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_train['tag'] = 'train'
print df_train.shape

In [ ]:
df_test = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/test.csv') 
df_test['tag'] = 'test'
print df_test.shape

In [ ]:
df = pd.concat([df_train,df_test],axis= 0)
print df.shape
df.head()

In [ ]:
df['Complaint-reason'] = df['Complaint-reason'].apply(preprocess_data)

In [ ]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

model = loadGloveModel('/home/prashant/Downloads/kagsa/data/glove.840B.300d.txt')
#vocab = model.keys()

In [ ]:
def get_vector(model,all_data):
    vector_sen = []
    for d in all_data:
        single_sen_vec = []
        words = d.split(' ')
        for w in words:
            try:
                
                get_word_vec = model[w]
                print w
            except:
                pass
            single_sen_vec.append(get_word_vec)
        v = np.array(single_sen_vec).mean(axis=0)
        vector_sen.append(v)
    return vector_sen

In [ ]:
vectors = get_vector(model,list(df['Complaint-reason']))

In [ ]:
# import time
# from sklearn.manifold import TSNE
# n_sne = 7000
# time_start = time.time()
# tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
# tsne_results = tsne.fit_transform(vectors)

In [ ]:
#ason"].map(str) + ' ' + df["Transaction-Type"]

In [ ]:
# text = list(df['Complaint-reason'])
# vectorizer = TfidfVectorizer(min_df=1)
# # tokenize and build vocab
# vectorizer.fit(text)

In [ ]:
# # encode document
# vector = vectorizer.transform(text)
# # summarize encoded vector
# print(vector.shape)
# #print(vector.toarray())
# vector

In [ ]:
vec_df = pd.DataFrame(vectors)

In [ ]:
print vec_df.shape
vec_df.head()


In [ ]:
vec_df['tag'] = list(df['tag'])
vec_df['Complaint-Status'] = list(df['Complaint-Status'])
#vec_df['Transaction-Type'] = list(df['Transaction-Type'])
vec_df['Complaint-ID'] = list(df['Complaint-ID'])
#vec_df['cd'] = list(df['Consumer-disputes'])

In [ ]:
#vec_df['cd'] = list(df['Consumer-disputes'])

In [ ]:
vec_df.head()

In [ ]:
complete_df_one_hot = vec_df#pd.get_dummies(data=vec_df, columns=['Transaction-Type'])

In [ ]:
complete_df_one_hot.head()

In [ ]:
train_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'train']
train_data.drop(['tag'],inplace=True,axis=1)
test_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'test']
test_data.drop(['tag'],inplace=True,axis =1)


In [ ]:
train_data.shape

In [ ]:
train_data.head()

In [ ]:
y_train = (train_data['Complaint-Status'])
x_train = train_data.drop(['Complaint-Status','Complaint-ID'],axis =1)

In [ ]:
# pd.concat([x_train,feature_tt])
x_train.shape

In [ ]:
print y_train.shape
y_train_one_hot = pd.get_dummies(y_train,prefix=['Complaint-Status'])
print y_train_one_hot.shape

In [ ]:
x_train.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,random_state = 0)


In [ ]:
print X_train.shape
print y_train_internal.shape

In [ ]:
from xgboost import XGBClassifier
xg = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='multi:softmax', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
gamma=10)

In [ ]:
xg.fit(X_train, y_train_internal)

In [ ]:
pred_internal_ = xg.predict(X_test)
import sklearn
sklearn.metrics.accuracy_score(y_test_internal, pred_internal_)

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
X = X_train
Y = y_train_internal
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 500
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
#results = model_selection.cross_val_score(model, X, Y, cv=kfold)
#print(results.mean())
model.fit(X_train, y_train_internal)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
model1 = LogisticRegression(random_state=1)
model2 = DecisionTreeClassifier(random_state=1)
model_ = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')
model_.fit(X_train, y_train_internal)


In [ ]:
f1_score(y_true, y_pred, average=None)

In [ ]:
pred_internal_ = model_.predict(X_test)
import sklearn
p = sklearn.metrics.f1_score(y_test_internal, pred_internal_,average='weighted')
p

In [ ]:
gb = MultinomialNB().fit(X_train, y_train_internal)
pred_internal = gb.predict(X_test)
import sklearn
sklearn.metrics.f1_score(y_test_internal, pred_internal,average='weighted')

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Train the model on training data
rm = rf.fit(X_train, y_train_internal)
pred_internal = rm.predict(X_test)



In [ ]:
import sklearn
sklearn.metrics.f1_score(y_test_internal, pred_internal,average='weighted')

In [ ]:
from sklearn.svm import SVC
sv = SVC(kernel='rbf')
sv.fit(X_train, y_train_internal)
pred_internal = sv.predict(X_test)

In [ ]:
import sklearn
sklearn.metrics.f1_score(y_test_internal, pred_internal,average = 'weighted')

In [ ]:
clf = MultinomialNB().fit(x_train, y_train)

In [ ]:
id_ = list(test_data['Complaint-ID'])


In [ ]:
test_data.head()
test_data.drop(['Complaint-Status','Transaction-Type','Complaint-ID'],inplace=True,axis=1)

In [ ]:
y_pred = clf.predict(test_data)

In [ ]:
import sklearn
sklearn.metrics.f1_score(y_test, y_pred,average='weighted')

In [ ]:
result = pd.DataFrame(id_,columns=['Complaint-ID'])
result['Complaint-Status'] = y_pred

In [ ]:
result.head()

In [ ]:
result.to_csv('/home/prashant/Downloads/c3cc8568-0-dataset/baseline.csv',index=None)